In [1]:
import argparse
import numpy as np
import sys
from os import path, stat, system
from datetime import datetime,timedelta
import matplotlib
matplotlib.use('agg')
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.cm as mplc
from matplotlib.colors import BoundaryNorm, ListedColormap
import xarray as xr
import os
import matplotlib.ticker as ticker

print("Ready!")


Ready!


In [2]:
parser = argparse.ArgumentParser(description="Aggregate partial sums and object distributions over a range of forecast cases.\nNOTE: All arguments listed below are REQUIRED, except for -diag",
                                 formatter_class=argparse.RawDescriptionHelpFormatter,
                                 usage="aggregate_MODE_scores.py [-h] [options]",
                                 epilog="SECOND of three scripts necessary to perform and plot object-based verification scores using MODE.")

arg_dict = {
    'model': 'RRFS_CONUS_3km',
    'field': 'compref',
    'dump_root': '/mnt/lfs4/BMC/amb-verif/RT_MODE/python_data',
    'idate': '2022051800',
    'edate': '2022070200',
    'diag': 1
}                                 

args = argparse.Namespace(**arg_dict)
                                 
# Apply input arguments to global settings
name = args.model
field = args.field
dump_dir = args.dump_root + '/' + name + '/' + field
img_root = "/lfs4/BMC/wrfruc/nickel/mode_img"
img_dir = "{}/{}/{}/images/valid_time".format(img_root,name,field)
iyear = int(args.idate[:4])
eyear = int(args.edate[:4])
imonth = int(args.idate[4:6])
emonth = int(args.edate[4:6])
iday = int(args.idate[6:8])
eday = int(args.edate[6:8])
ihr = int(args.idate[8:10])
ehr = int(args.edate[8:10])
os.makedirs(img_dir, exist_ok = True)

# Hard-coded constants/settings
fcst_length = 3
tod_start = 6
T_start = 1
delta_hr = 1 # distance between forecast initialization times
if field == "compref":
   accum_hr = "00"
elif field == "precip":
   accum_hr = "06"
n_rad = 1
n_thresh = 4
match_int = 0.70 # threshold interest value to declare a match
agg_vhr_start = [0,6,12,18,0,6,21]
agg_vhr_end =   [5,11,17,23,23,17,3]
dx = 3.0 # grid spacing
match_int = 0.70 # threshold interest value to declare a match
# create color scheme for multiple convolution radius and threshold testing
centroid_FB_plot = True
cmp = mplc.get_cmap('coolwarm')
nq = np.linspace(0,1,n_rad*n_thresh+1)
colors = cmp(nq)

###################### END SETTINGS #######################################

print("Done!")

Done!


In [3]:
def calc_CRPS(F,O,data_bins):
   delta = data_bins[1:] - data_bins[:-1]
   f_hist,bins = np.histogram(F,bins=data_bins)
   o_hist,bins = np.histogram(O,bins=data_bins)
   f_hist = f_hist / float(len(F))
   o_hist = o_hist / float(len(O))
   score = np.sum(delta*(np.cumsum(f_hist) - np.cumsum(o_hist))**2)
   return score

def fmt(x, pos):
    a, b = '{:.1e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

end_time = datetime(year=eyear,month=emonth,day=eday,hour=ehr)

# map projections
# full CONUS
if False:
   clat = 39.0
   clon = -95.0
   proj_wid = 5.25e6
   proj_hgt = 3.25e6
   min_area = 500
   res = 'h'
   lbl_off_x = 30000
   lbl_off_y = 20000
   figsize = (12,8)
else:
# Eastern 2/3ds US
   clat = 39.0
   clon = -90.0
   proj_wid = 3.05e6
   proj_hgt = 2.45e6
   min_area = 250
   res = 'h'
   lbl_off_y = 10000
   lbl_off_x = 15000
   figsize = (12,9.5)

# set lat and lon bins for centroid binning
lon_bins = np.arange(-110.0,-65.1,1.0)
lat_bins = np.arange(25.0,51.1,1.0)
res_label = "1p0"
lats_2d,lons_2d = np.meshgrid(lat_bins,lon_bins)
# set x-/y- centroid error bins [km...raw data is in m...need to convert]
x_bins = np.arange(-150.0,150.1,10.0)
y_bins = np.arange(-150.0,150.1,10.0)
cy2d,cx2d = np.meshgrid(y_bins,x_bins)

area_bins = np.array([144,200,300,400,500,600,800,1000,1250,1500,2000,2500,3000,4000,5000,10000,20000,50000])
mass_bins = np.array([0.0,0.1,1,2,4,8,16,32,64,100,200,500,1000,1500,2000,3000,5000])
if field == "precip":
   pXX_bins = np.array([0,1,3,5,10,15,20,25,30,40,50,100])
elif field == "compref":
   pXX_bins = np.arange(20.,75.1,5.0)
curvature_bins = np.arange(700.,3500.1,100.)

for R in range(1,n_rad+1):
   print("WORKING ON RADIUS {}".format(R))
   for T in range(T_start,n_thresh+1):
      print("WORKING ON THRESHOLD {}".format(T))

      # to set color index in later plots
      color_number = n_thresh*(R-1) + T - 1 # to force 0-base

      # Output metrics as a function of time of day
      # aggregated across all cases
      num_fcst_objs = np.zeros(24,dtype=np.int)
      num_obs_objs = np.zeros(num_fcst_objs.shape,dtype=np.int)
      num_pairs = np.zeros(num_fcst_objs.shape,dtype=np.int)
      OTS = np.full(num_fcst_objs.shape,-999.,dtype=np.float)
      MMI = np.full(num_fcst_objs.shape,-999.,dtype=np.float)
      total_pod = np.full(num_fcst_objs.shape,-999.,dtype=np.float)
      total_sr = np.full(num_fcst_objs.shape,-999.,dtype=np.float)
      total_far = np.full(num_fcst_objs.shape,-999.,dtype=np.float)
      total_csi = np.full(num_fcst_objs.shape,-999.,dtype=np.float)
      mean_centroid_dist = np.full((24,5),-999.,dtype=np.float)
      median_centroid_dist = np.full(mean_centroid_dist.shape,-999.,dtype=np.float)
      std_centroid_dist = np.zeros(mean_centroid_dist.shape,dtype=np.float)
      fbias = -999.

      # All-24-hour aggregated object attribute distributions
      all_f_area = np.empty(0,dtype=np.int)
      all_f_aspect = np.empty(0,dtype=np.float)
      all_f_complexity = np.empty(0,dtype=np.float)
      all_f_pXX = np.empty(0,dtype=np.float)
      all_f_mass = np.empty(0,dtype=np.float)
      all_o_area = np.empty(0,dtype=np.int)
      all_o_aspect = np.empty(0,dtype=np.float)
      all_o_complexity = np.empty(0,dtype=np.float)
      all_o_pXX = np.empty(0,dtype=np.float)
      all_o_mass = np.empty(0,dtype=np.float)

      # Based on this array dimensionality, aggregated scores cannot be plotted in this code and must instead be plotted in the "plot_agg_MODE_scores.py" script
      # Final time-aggregated scores
      agg_OTS = np.full((len(agg_vhr_start)),-999.,dtype=np.float)
      agg_MMI = np.full(agg_OTS.shape,-999.,dtype=np.float)
      agg_pod = np.full(agg_OTS.shape,-999.,dtype=np.float)
      agg_far = np.full(agg_OTS.shape,-999.,dtype=np.float)
      agg_sr = np.full(agg_OTS.shape,-999.,dtype=np.float)
      agg_csi = np.full(agg_OTS.shape,-999.,dtype=np.float)
      agg_mean_dist = np.full(agg_OTS.shape,-999.,dtype=np.float) # ONLY ALL MATCHED objects (not JUST STORMS)
      agg_gen_dist = np.full(agg_OTS.shape,-999.,dtype=np.float) # for generalized distance
      agg_fcst_count = np.zeros(agg_OTS.shape,dtype=np.int)
      agg_obs_count = np.zeros(agg_OTS.shape,dtype=np.int)
      agg_fbias = np.full(agg_OTS.shape,np.nan,dtype=np.float) # object count

      # Time-aggregated score components
      agg_ots_sum = np.zeros((len(agg_vhr_start),2),dtype=np.float)
      # One agg_max_int array for each possible set of aggregated time periods
      for d in range(0,len(agg_vhr_start)):
         exec("agg_f_cent_lon_{:02d} = np.empty(0,dtype=np.float)".format(d))
         exec("agg_f_cent_lat_{:02d} = np.empty(0,dtype=np.float)".format(d))
         exec("agg_o_cent_lon_{:02d} = np.empty(0,dtype=np.float)".format(d))
         exec("agg_o_cent_lat_{:02d} = np.empty(0,dtype=np.float)".format(d))
         exec("agg_max_int_{:02d} = np.empty(0,dtype=np.float)".format(d))
         exec("agg_gen_dist_{:02d} = np.empty(0,dtype=np.float)".format(d))
      agg_hit = np.zeros(len(agg_vhr_start),dtype=np.int)
      agg_fa = np.zeros(len(agg_vhr_start),dtype=np.int)
      agg_miss = np.zeros(len(agg_vhr_start),dtype=np.int)
      agg_dist_sum = np.zeros(len(agg_vhr_start),dtype=np.float)
      agg_dist_count = np.zeros(len(agg_vhr_start),dtype=np.int)

      ff = 0 # ff will still correspond to time-of-day index
      if T != T_start:
         tod_start = 6
      for tod in [6, 18]:
         time_of_day = "{:02d}".format(tod)
         print("WORKING ON TIME OF DAY {}00 UTC".format(time_of_day))

         ots_sum = np.zeros(2,dtype=np.float)
         max_int_array = np.empty(0,dtype=np.float)

         # initialize empty arrays for all-case distribution. Since we don't know the number of forecast and observation objects
         # at the moment we first perform the data read, we will start with empty arrays and build them up as we go.
         ### individual object qualities
         f_cent_lat = np.empty(0,dtype=np.float)
         f_cent_lon = np.empty(0,dtype=np.float)
         f_angle = np.empty(0,dtype=np.float)
         f_length = np.empty(0,dtype=np.float)
         f_width = np.empty(0,dtype=np.float)
         f_aspect = np.empty(0,dtype=np.float)
         f_area = np.empty(0,dtype=np.float)
         f_curvature = np.empty(0,dtype=np.float)
         f_complexity = np.empty(0,dtype=np.float)
         f_p10 = np.empty(0,dtype=np.float)
         f_p25 = np.empty(0,dtype=np.float)
         f_p50 = np.empty(0,dtype=np.float)
         f_p75 = np.empty(0,dtype=np.float)
         f_p90 = np.empty(0,dtype=np.float)
         f_pXX = np.empty(0,dtype=np.float)
         f_mass = np.empty(0,dtype=np.float)
         o_cent_x = np.empty(0,dtype=np.float)
         o_cent_y = np.empty(0,dtype=np.float)
         o_cent_lat = np.empty(0,dtype=np.float)
         o_cent_lon = np.empty(0,dtype=np.float)
         o_angle = np.empty(0,dtype=np.float)
         o_length = np.empty(0,dtype=np.float)
         o_width = np.empty(0,dtype=np.float)
         o_aspect = np.empty(0,dtype=np.float)
         o_area = np.empty(0,dtype=np.float)
         o_area_t = np.empty(0,dtype=np.float)
         o_curvature = np.empty(0,dtype=np.float)
         o_complexity = np.empty(0,dtype=np.float)
         o_p10 = np.empty(0,dtype=np.float)
         o_p25 = np.empty(0,dtype=np.float)
         o_p50 = np.empty(0,dtype=np.float)
         o_p75 = np.empty(0,dtype=np.float)
         o_p90 = np.empty(0,dtype=np.float)
         o_pXX = np.empty(0,dtype=np.float)
         o_mass = np.empty(0,dtype=np.float)
         # object pair attributes
         pair_dcentroid = np.empty(0,dtype=np.float)
         pair_dangle = np.empty(0,dtype=np.float)
         pair_daspect = np.empty(0,dtype=np.float)
         pair_area_ratio = np.empty(0,dtype=np.float)
         pair_int_area = np.empty(0,dtype=np.float)
         pair_union_area = np.empty(0,dtype=np.float)
         pair_sym_diff_area = np.empty(0,dtype=np.float)
         pair_consum_ratio = np.empty(0,dtype=np.float)
         pair_curv_ratio = np.empty(0,dtype=np.float)
         pair_complex_ratio = np.empty(0,dtype=np.float)
         pair_pct_intense_ratio = np.empty(0,dtype=np.float)
         pair_interest = np.empty(0,dtype=np.float)
         # contingency table statistics
         hit = 0
         miss = 0
         false_alarm = 0
         n_matches = 0
         storm_match_dist_array = np.empty(0,dtype=np.float)
         all_match_dist_array = np.empty(0,dtype=np.float)
         gen_match_dist_array = np.empty(0,dtype=np.float)
         gen_x_error_match_dist = np.empty(0,dtype=np.float)
         gen_y_error_match_dist = np.empty(0,dtype=np.float)

         n_cases = 0
         # Determine the number of forecast lengths to evaluate
         if tod % delta_hr == 0:
            N_fhr_check = 1 + (fcst_length/delta_hr)
         else:
            N_fhr_check = fcst_length/delta_hr
         for n in range(1,int(N_fhr_check)+1):
            # Finally, the magical formula that does this right!
            fhr = (tod % delta_hr + (n-1)*delta_hr)
            init_hr = (tod - fhr) % 24
            lead = "{:02d}".format(fhr)
            
            # loop over all cases/forecast files starting at "init_hr" with forecast hour "fhr"
            # time is the time at the start of each forecast case
            if fhr > 0:
                start_time = datetime(year=iyear,month=imonth,day=iday,hour=init_hr)
                time = start_time
                if args.diag >= 1:
                   print("looping over all {:02d}-hour forecasts starting at {:02d}Z...".format(fhr,init_hr))
                while time <= end_time:
                   cyear = str(time.year)
                   cmonth = "{:02d}".format(time.month)
                   cday = "{:02d}".format(time.day)
                   chour = "{:02d}".format(time.hour)
                   casedir = cyear + cmonth + cday + chour
                   # set up the valid time
                   vtime = time + timedelta(hours=fhr)
                   vyear = str(vtime.year)
                   vmonth = "{:02d}".format(vtime.month)
                   vday = "{:02d}".format(vtime.day)
                   vhour = "{:02d}".format(vtime.hour)

                   if args.diag >= 2:
                      print("WORKING ON FORECAST CASE {}{}{} {}Z".format(cyear,cmonth,cday,chour))

                   psum_filename = "{}/partial_sums/{}/partial_sums_r{}t{}_{}_f{}.npz".format(dump_dir,casedir,R,T,casedir,lead)
                   attr_filename = "{}/object_attributes/{}/attributes_r{}t{}_{}_f{}.npz".format(dump_dir,casedir,R,T,casedir,lead)
                   if path.isfile(psum_filename) and path.isfile(attr_filename):
                      data_psum = np.load(psum_filename)
                      data_attr = np.load(attr_filename)
                      n_cases += 1
                   else:
                      if args.diag >= 2:
                         print(" partial sum file {} not found!".format(psum_filename))
                      # Increment time and move to next case
                      time += timedelta(hours=24)
                      continue

                   # Construct all-case sum from partial sums
                   ots_sum[0] += data_psum['ots_sum_numer']
                   ots_sum[1] += data_psum['ots_sum_denom']
                   num_fcst_objs[ff] += data_psum['n_f_objs']
                   num_obs_objs[ff] += data_psum['n_o_objs']
                   hit += data_psum['n_hit']
                   miss += data_psum['n_miss']
                   false_alarm += data_psum['n_fa']

                   # Construct large arrays to make final calculations
                   max_int_array = np.append(max_int_array,data_attr['max_interest'])
                   #standard_MMI = data_attr['MMI_flag']
                   standard_MMI = True
                   all_match_dist_array = np.append(all_match_dist_array,data_attr['all_match_dist'])
                   storm_match_dist_array = np.append(storm_match_dist_array,data_attr['storm_match_dist'])
                   gen_match_dist_array = np.append(gen_match_dist_array,data_attr['gen_match_dist'])
                   gen_x_error_match_dist = np.append(gen_x_error_match_dist,data_attr['gen_match_x_error'])
                   gen_y_error_match_dist = np.append(gen_y_error_match_dist,data_attr['gen_match_y_error'])

                   # Append storm attribute file arrays to all-case arrays
                   f_cent_lat = np.append(f_cent_lat,data_attr['file_f_cent_lat'])
                   f_cent_lon = np.append(f_cent_lon,data_attr['file_f_cent_lon'])
                   f_angle = np.append(f_angle,data_attr['file_f_angle'])
                   f_length = np.append(f_length,data_attr['file_f_length'])
                   f_width = np.append(f_width,data_attr['file_f_width'])
                   f_aspect = np.append(f_aspect,data_attr['file_f_aspect'])
                   f_area = np.append(f_area,data_attr['file_f_area'])
                   f_curvature = np.append(f_curvature,data_attr['file_f_curvature'])
                   f_complexity = np.append(f_complexity,data_attr['file_f_complexity'])
                   f_p10 = np.append(f_p10,data_attr['file_f_p10'])
                   f_p25 = np.append(f_p25,data_attr['file_f_p25'])
                   f_p50 = np.append(f_p50,data_attr['file_f_p50'])
                   f_p75 = np.append(f_p75,data_attr['file_f_p75'])
                   f_p90 = np.append(f_p90,data_attr['file_f_p90'])
                   f_pXX = np.append(f_pXX,data_attr['file_f_pXX'])
                   f_mass = np.append(f_mass,data_attr['file_f_mass'])
                   o_cent_lat = np.append(o_cent_lat,data_attr['file_o_cent_lat'])
                   o_cent_lon = np.append(o_cent_lon,data_attr['file_o_cent_lon'])
                   o_angle = np.append(o_angle,data_attr['file_o_angle'])
                   o_length = np.append(o_length,data_attr['file_o_length'])
                   o_width = np.append(o_width,data_attr['file_o_width'])
                   o_aspect = np.append(o_aspect,data_attr['file_o_aspect'])
                   o_area = np.append(o_area,data_attr['file_o_area'])
                   o_curvature = np.append(o_curvature,data_attr['file_o_curvature'])
                   o_complexity = np.append(o_complexity,data_attr['file_o_complexity'])
                   o_p10 = np.append(o_p10,data_attr['file_o_p10'])
                   o_p25 = np.append(o_p25,data_attr['file_o_p25'])
                   o_p50 = np.append(o_p50,data_attr['file_o_p50'])
                   o_p75 = np.append(o_p75,data_attr['file_o_p75'])
                   o_p90 = np.append(o_p90,data_attr['file_o_p90'])
                   o_pXX = np.append(o_pXX,data_attr['file_o_pXX'])
                   o_mass = np.append(o_mass,data_attr['file_o_mass'])
                   pair_dcentroid = np.append(pair_dcentroid,data_attr['file_pair_dcentroid'])
                   pair_dangle = np.append(pair_dangle,data_attr['file_pair_dangle'])
                   pair_daspect = np.append(pair_daspect,data_attr['file_pair_daspect'])
                   pair_area_ratio = np.append(pair_area_ratio,data_attr['file_pair_area_ratio'])
                   pair_int_area = np.append(pair_int_area,data_attr['file_pair_int_area'])
                   pair_union_area = np.append(pair_union_area,data_attr['file_pair_union_area'])
                   pair_sym_diff_area = np.append(pair_sym_diff_area,data_attr['file_pair_sym_diff_area'])
                   pair_consum_ratio = np.append(pair_consum_ratio,data_attr['file_pair_consum_ratio'])
                   pair_curv_ratio = np.append(pair_curv_ratio,data_attr['file_pair_curv_ratio'])
                   pair_complex_ratio = np.append(pair_complex_ratio,data_attr['file_pair_complex_ratio'])
                   pair_pct_intense_ratio = np.append(pair_pct_intense_ratio,data_attr['file_pair_pct_intense_ratio'])
                   pair_interest = np.append(pair_interest,data_attr['file_pair_interest'])

               #    print("There were {} forecast objects, {} observation objects, and {} pairs to evaluate in this file".format(n_f,n_o,n_pairs))

                   time += timedelta(hours=24)

            # END case loop (forecast init/cycle time)
         # END loop through forecast-hours-to-check

         num_fcst_objs[ff] = len(f_cent_lon)
         num_obs_objs[ff] = len(o_cent_lon)
         num_pairs[ff] = len(pair_dcentroid)

         # Aggregate object attributes over all hours of the day
         all_f_area = np.append(all_f_area,f_area)
         all_f_aspect = np.append(all_f_aspect,f_aspect)
         all_f_complexity = np.append(all_f_complexity,f_complexity)
         all_f_pXX = np.append(all_f_pXX,f_pXX)
         all_f_mass = np.append(all_f_mass,f_mass)
         all_o_area = np.append(all_o_area,o_area)
         all_o_aspect = np.append(all_o_aspect,o_aspect)
         all_o_complexity = np.append(all_o_complexity,o_complexity)
         all_o_pXX = np.append(all_o_pXX,o_pXX)
         all_o_mass = np.append(all_o_mass,o_mass)

         for d in range(0,len(agg_vhr_start)):
            # Typical range for aggregation period vs. aggregation period spanning 00Z
            if ((agg_vhr_start[d] < agg_vhr_end[d]) and (tod >= agg_vhr_start[d] and tod <= agg_vhr_end[d])) \
            or \
            ((agg_vhr_start[d] > agg_vhr_end[d]) and (tod >= agg_vhr_start[d] or tod <= agg_vhr_end[d])):
               exec("agg_f_cent_lon_{:02d} = np.append(agg_f_cent_lon_{:02d},f_cent_lon)".format(d,d))
               exec("agg_f_cent_lat_{:02d} = np.append(agg_f_cent_lat_{:02d},f_cent_lat)".format(d,d))
               exec("agg_o_cent_lon_{:02d} = np.append(agg_o_cent_lon_{:02d},o_cent_lon)".format(d,d))
               exec("agg_o_cent_lat_{:02d} = np.append(agg_o_cent_lat_{:02d},o_cent_lat)".format(d,d))
               exec("agg_gen_dist_{:02d} = np.append(agg_gen_dist_{:02d},gen_match_dist_array)".format(d,d))

         if args.diag >= 1:
            print("There are a total of {} forecast objects and {} observation objects to evaluate over {} cases at {:02d}00 UTC".format(num_fcst_objs[ff],num_obs_objs[ff],n_cases,tod))
            print("There were {} matched object pairs representing the shape of a single intense thunderstorm evaluated at {:02d}00 UTC".format(len(storm_match_dist_array),tod))

         if len(storm_match_dist_array) > 0:
            mean_centroid_dist[ff,0] = np.mean(storm_match_dist_array)
            median_centroid_dist[ff,0] = np.median(storm_match_dist_array)
            std_centroid_dist[ff,0] = np.std(storm_match_dist_array)
         if len(all_match_dist_array) > 0:
            mean_centroid_dist[ff,1] = np.mean(all_match_dist_array)
            median_centroid_dist[ff,1] = np.median(all_match_dist_array)
            std_centroid_dist[ff,1] = np.std(all_match_dist_array)
         if len(gen_match_dist_array) > 0:
            mean_centroid_dist[ff,2] = np.mean(gen_match_dist_array)
            median_centroid_dist[ff,2] = np.median(gen_match_dist_array)
            std_centroid_dist[ff,2] = np.std(gen_match_dist_array)

         # Add time-aggregated score components to arrays
         # (The final calculation of time-aggregated scores must occur outside of the forecast-hour loop...so...in other words...outside of THIS loop)
         for d in range(0,len(agg_vhr_start)):
            # Typical range for aggregation period vs. aggregation period spanning 00Z
            if ((agg_vhr_start[d] < agg_vhr_end[d]) and (tod >= agg_vhr_start[d] and tod <= agg_vhr_end[d])) \
            or \
            ((agg_vhr_start[d] > agg_vhr_end[d]) and (tod >= agg_vhr_start[d] or tod <= agg_vhr_end[d])):
               agg_ots_sum[d,0] += ots_sum[0]
               agg_ots_sum[d,1] += ots_sum[1]
               exec("agg_max_int_{:02d} = np.append(agg_max_int_{:02d},max_int_array)".format(d,d))
               exec("agg_gen_dist_{:02d} = np.append(agg_gen_dist_{:02d},gen_match_dist_array)".format(d,d))
               agg_dist_sum[d] += np.sum(all_match_dist_array)
               agg_dist_count[d] += len(all_match_dist_array)
               agg_hit[d] += hit
               agg_miss[d] += miss
               agg_fa[d] += false_alarm
               agg_fcst_count[d] += num_fcst_objs[ff]
               agg_obs_count[d] += num_obs_objs[ff]
         for d in range(0,len(agg_vhr_start)):
            exec("agg_gen_dist[{}] = np.mean(agg_gen_dist_{:02d})".format(d,d))
            if agg_obs_count[d] > 0:
               agg_fbias[d] = float(agg_fcst_count[d]) / agg_obs_count[d]

         if num_fcst_objs[ff] > 0 and num_obs_objs[ff] > 0:

            if hit + miss > 0:
               total_pod[ff] = float(hit) / (hit + miss)
            if hit + false_alarm > 0:
               total_sr[ff] = float(hit) / (hit + false_alarm)
               total_far[ff] = float(false_alarm) / (hit + false_alarm)
            if hit + miss + false_alarm > 0:
               total_csi[ff] = float(hit) / (hit + miss + false_alarm)

            OTS[ff] = ots_sum[0] / ots_sum[1]
            MMI[ff] = np.median(max_int_array)

            # Calculate object attribute distribution CRPSs
            cent_lon_crps = calc_CRPS(f_cent_lon,o_cent_lon,np.arange(-108.,-75.1,0.1))
            cent_lat_crps = calc_CRPS(f_cent_lat,o_cent_lat,np.arange(25.0,51.1,0.1))
            area_crps = calc_CRPS(f_area,o_area,area_bins)
            length_crps = calc_CRPS(f_length,o_length,np.arange(2.0,751.0,5.0))
            width_crps = calc_CRPS(f_width,o_width,np.arange(1,301,1.0))
            aspect_crps = calc_CRPS(f_aspect,o_aspect,np.arange(0.0,1.01,0.01))
            complex_crps = calc_CRPS(f_complexity,o_complexity,np.arange(0.0,1.01,0.01))
            pXX_crps = calc_CRPS(f_pXX,o_pXX,pXX_bins)
            curv_crps = calc_CRPS(f_curvature,o_curvature,np.arange(0,3001,10))

            if False:
               # What is the typical area of objects with a given curvature?
               curvature_bins = np.arange(700.,3500.1,100.)
               curvature_area_sum = np.zeros_like(curvature_bins,dtype=np.float)
               curvature_num = np.zeros_like(curvature_bins,dtype=np.int)
               mean_area_curv = np.zeros_like(curvature_bins,dtype=np.float)
               curvature_ar_sum = np.zeros_like(curvature_bins,dtype=np.float)
               mean_AR_curv = np.zeros_like(curvature_bins,dtype=np.float)
               for a in range(0,num_fcst_objs[ff]):
                  for b in range(0,len(curvature_bins)-1):
                     if f_curvature[a] > curvature_bins[b] and f_curvature[a] <= curvature_bins[b+1]:
                        curvature_area_sum[b] += f_area[a]
                        curvature_num[b] += 1
                        curvature_ar_sum[b] += f_aspect[a]
                        break
               mean_area_curv = curvature_area_sum / curvature_num
               mean_AR_curv = curvature_ar_sum / curvature_num
               f = plt.figure(figsize=(5,5))
               ax1 = f.add_axes([0.125,0.1,0.775,0.88])
               ax1.set_xlim(curvature_bins[0],curvature_bins[-1])
               ax1.set_xticks(curvature_bins[::2])
               ax1.set_xlabel('Curvature value',fontsize=9)
               ax2 = plt.twinx(ax1)
               h1 = ax1.plot(0.5*(curvature_bins[:-1] + curvature_bins[1:]),mean_area_curv[:-1],'r-x',linewidth=2,label="area")
               h2 = ax2.plot(0.5*(curvature_bins[:-1] + curvature_bins[1:]),mean_AR_curv[:-1],'b-x',linewidth=2,label="aspect ratio")
               for a in range(0,len(curvature_bins)-1):
                  if curvature_num[a] > 0:
                     ax1.text(0.5*(curvature_bins[a] + curvature_bins[a+1]),mean_area_curv[a]+25,"{:4d}".format(curvature_num[a]),ha='center',va='bottom',fontsize=6,fontweight=200,rotation=45)
               ax1.grid(linestyle=":",color="0.75")
               ax2.set_xlim(curvature_bins[0],curvature_bins[-1])
               ax2.set_ylim(0,1)
               ax2.set_yticks(np.arange(0.,1.01,0.1))
               ax2.set_ylabel("aspect ratio",fontsize=9)
               ax1.set_ylabel(r"object area [$km^2$]",fontsize=9)
               ax1.tick_params(axis='both',labelsize=6)
               ax2.tick_params(axis='y',labelsize=6)
               lns = h1 + h2
               lbls = [l.get_label() for l in lns]
               ax1.legend(lns,lbls,loc=0,fontsize=8)
               image_file = "{}/{}_curvature_parameters_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               f.savefig(image_file,dpi=120)
               f.close()

            if True:
               # Determine object count bias as a function of object size
               f_hist,bin = np.histogram(dx**2*f_area,bins=area_bins)
               o_hist,bin = np.histogram(dx**2*o_area,bins=area_bins)
               bias_by_size = 1.0*f_hist / o_hist
               # Also corroborate frequency (coverage) bias from MATS by calculating total area
               fbias = np.sum(f_area) / np.sum(o_area)
               plt.figure(figsize=(5,5))
               plt.subplots_adjust(left=0.125,bottom=0.1,top=0.7,right=0.95)   #top=0.98
               plt.semilogx(0.5*(bin[:-1]+bin[1:]),bias_by_size,'k-x',linewidth=1,label=name)
               plt.grid(linestyle=":",color='0.75')
               if T <= 2:
                  plt.ylim(0.5,2.25)
               else:
                  plt.ylim(0.9,30)
                  plt.yscale('log')
               plt.title("{}Z MODE Object Bias by Size \n 18 May 2022 - 02 July 2022".format(time_of_day), fontsize = 12)
               plt.xlabel(r"Object Area [km$^{2}$]",size=9)
               plt.ylabel("Object Count Bias (F/O)",size=9)
               plt.tick_params(axis='both',labelsize=8)
               plt.axvspan(0, 1000, alpha=0.5, color='lightgreen')
               plt.legend(loc=1,prop={'size':9})
               image_file = "{}/{}_object_bias_by_size_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

            if True:
               # map of east-west and south-north components of centroid error
               plt.figure(figsize=(5.5,5))
               plt.subplots_adjust(left=0.125,bottom=0.1,top=0.80,right=0.875)
               centroid_error_hist2d,binx,biny = np.histogram2d(gen_x_error_match_dist/1000.,gen_y_error_match_dist/1000.,bins=[x_bins,y_bins]) # division is to convert from m to km
               hist = centroid_error_hist2d / len(gen_match_dist_array)
               clevs = [1e-5,1e-4,2.5e-4,5e-4,7.5e-4,1e-3,2.5e-3,5e-3,7.5e-3,1e-2,2.5e-2,5e-2,7.5e-2,1e-1,2e-1]
               ccs = mplc.get_cmap('BuPu',len(clevs))
               plt.contourf(0.5*(cx2d[:-1,:-1]+cx2d[1:,1:]),0.5*(cy2d[:-1,:-1]+cy2d[1:,1:]),hist,clevs,colors=ccs(range(len(clevs))),extend='both')
               plt.grid(linestyle=":",color='0.75')
               plt.xlim(x_bins[0],x_bins[-1])
               plt.ylim(y_bins[0],y_bins[-1])
               plt.xlabel("West-East Centroid Error [km]",size= 9)
               plt.ylabel("South-North Centroid Error [km]",size= 9)
               plt.tick_params(axis='both',labelsize=8)
               plt.title("{}Z MODE Centroid Error".format(time_of_day), fontsize = 12)
               cb = plt.colorbar(orientation='vertical',fraction=0.05,aspect=20,pad=0.03,shrink=0.75,extend='max',format=ticker.FuncFormatter(fmt))
               cb.set_ticks(clevs)
               cb.set_label("relative frequency",fontsize=10)
               cb.ax.tick_params(labelsize=6)
               plt.plot(np.nanmean(gen_x_error_match_dist)/1000.,np.nanmean(gen_y_error_match_dist)/1000.,'xk',ms=6,mew=2,label="mean error location")
               plt.plot(np.nanmedian(gen_x_error_match_dist)/1000.,np.nanmedian(gen_y_error_match_dist)/1000.,'ok',ms=6,mew=2,label="median error location")
               plt.legend(loc='upper right',prop={'size':8})
               image_file = "{}/{}_object_centroid_error_map_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

            if True:
               # Projected map plots of object centroids
               # We will keep this code so that any of the following three types of plots can be made just by commenting or setting flags (rather than replacing the code altogether)
               # - forecast-object centroid density
               # - observation-object centroid density
               # - frequency bias of object centroid density
               f_hist2d,binx,biny = np.histogram2d(f_cent_lon,f_cent_lat,bins=[lon_bins,lat_bins])
               o_hist2d,binx,biny = np.histogram2d(o_cent_lon,o_cent_lat,bins=[lon_bins,lat_bins])
               print(np.sum(f_hist2d))
               print(np.sum(o_hist2d))
               if centroid_FB_plot:
                fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
               else:
                hist2d = f_hist2d / (1.0*num_fcst_objs[ff])
                #hist2d = o_hist2d / (1.0*num_obs_objs[ff])
                hist_max = 10**np.ceil(np.log10(np.amax(hist2d)))
               fig = plt.figure(figsize=figsize)
               ax = plt.subplots_adjust(left=0.01,right=0.95,bottom=0.05,top=0.97)
               m = Basemap(projection='lcc',width=proj_wid,height=proj_hgt,lat_0=clat,lon_0=clon,lat_1=38.5,resolution=res,area_thresh=min_area)
               m.drawlsmask(land_color='0.9',ocean_color='powderblue')
               m.drawcountries(linewidth=1,color='0.1')
               if True:
                  m.drawcoastlines(linewidth=0.5,color='0.4')
               else:
                  m.drawcounties(linewidth=0.5,color='0.6',linestyle='-')
               m.drawstates(linewidth=0.5,color='0.5')
               m.drawmapboundary(color=[0.25,0.01,0.02],linewidth=3)
               m.drawmeridians(np.arange(-130.0,-50.1,5.0),linewidth=0.1,labels=[0,0,0,1],labelstyle='E/W',yoffset=lbl_off_y,fontsize=8,dashes=[2,2])
               m.drawparallels(np.arange(20.0,55.1,5.0),linewidth=0.1,labels=[0,1,0,0],labelstyle='E/W',xoffset=lbl_off_x,fontsize=8,dashes=[2,2])
               x2d,y2d = m(lons_2d,lats_2d)
               # Frequency bias plot
               if centroid_FB_plot:
                fb_clevs_low = np.array([0,0.25,0.5,0.75,0.9])
                if T <= 2:
                 fb_clevs_high = np.array([1.1,1.25,1.5,2,2.5,3,3.5,4])
                elif T == 3:
                 fb_clevs_high = np.array([1.1,1.25,1.5,2,2.5,3,3.5,4,5])
                elif T >= 4:
                 fb_clevs_high = np.array([1.1,1.25,1.5,2,2.5,3,3.5,4,5,7.5])
                fb_clevs = np.append(fb_clevs_low,fb_clevs_high)
                cm_low = mplc.get_cmap('Oranges_r',len(fb_clevs_low))
                cm_high = mplc.get_cmap('Purples',len(fb_clevs_high))
                cmap = np.vstack((cm_low(np.linspace(0,1,len(fb_clevs_low)-1)),np.array([0.9,1.0,0.9,1.0]),cm_high(np.linspace(0,1,len(fb_clevs_high)))))
                final_colors = ListedColormap(cmap)
                norm = BoundaryNorm(boundaries=fb_clevs,ncolors=len(fb_clevs),clip=True)
              #  plt.pcolormesh(x2d[:-1,:-1],y2d[:-1,:-1],fbias_hist2d,norm=norm,cmap=final_colors)
                plt.contourf(x2d[:-1,:-1],y2d[:-1,:-1],fbias_hist2d,levels=fb_clevs,colors=cmap,extend='max')
                cb = plt.colorbar(orientation='horizontal',fraction=0.05,aspect=50,pad=0.03,shrink=0.5,extend='max')
                cb.set_ticks(fb_clevs) # Frequency bias plot
                cb.set_label(r'Frequency bias ($N_f / N_o$)',fontsize=10)
                image_file = "{}/{}_centroid_fbias_heatmap_r{}t{}_{:02d}Z_{}.png".format(img_dir,name,R,T,tod,res_label)
               else:
                # Linear color scaling
               # plt.pcolormesh(x2d[:-1,:-1],y2d[:-1,:-1],hist2d,vmin=1,vmax=5000),cmap=mplc.get_cmap('PuRd'))
                # Logarithmic color scaling
                plt.pcolormesh(x2d[:-1,:-1],y2d[:-1,:-1],hist2d,norm=matplotlib.colors.LogNorm(vmin=1e-4,vmax=0.01),cmap=mplc.get_cmap('PuRd'))
                cb = plt.colorbar(orientation='horizontal',fraction=0.05,aspect=50,pad=0.03,shrink=0.5,extend='both')
                cb.set_ticks([1e-4,5e-4,1e-3,5e-3,0.01,0.05,0.1]) # For log-based normalization of this plot
                #cb.set_ticks([1,5,10,25,50,75,100,250,500,750,1000,5000])  # For linear scaling of this plot
                cb.set_label("relative frequency",fontsize=10)
                image_file = "{}/{}_centroid_heatmap_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               cb.ax.tick_params(labelsize=8)
               fig.savefig(image_file,dpi=120)
               plt.close()

            ### Object attribute distributions
            if True:
               # Object centroid distribution
               plt.figure(figsize=(9,5))
               plt.subplots_adjust(left=0.075,bottom=0.075,top=0.98,right=0.98,wspace=0.2)
               plt.subplot(1,2,1)
               f_hist,bin = np.histogram(f_cent_lon,bins=np.arange(-107.5,-79.51,1.0))
               o_hist,bin = np.histogram(o_cent_lon,bins=np.arange(-107.5,-79.51,1.0))
               plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),align='center',width=1.0,color='red',edgecolor='white',linewidth=1,label=name)
               plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),align='center',width=1.0,color='None',edgecolor='black',linewidth=1,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel("Object centroid longitude (deg.)",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               plt.legend(loc=1,prop={'size':8})
               mu_f = np.mean(f_cent_lon)
               mu_o = np.mean(o_cent_lon)
               std_f = np.std(f_cent_lon)
               std_o = np.std(o_cent_lon)
               med_f = np.median(f_cent_lon)
               med_o = np.median(o_cent_lon)
               plt.figtext(0.10,0.83,r"$\mu_f$={:5.1f};  $\mu_o$={:5.1f}".format(mu_f,mu_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.10,0.78,r"median$_f$={:5.1f};  median$_o$={:5.1f}".format(med_f,med_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.10,0.73,r"$\sigma_f$={:4.1f};  $\sigma_o$={:4.1f}".format(std_f,std_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.10,0.68,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.10,0.63,"CRPS={:5.3f}".format(cent_lon_crps),ha='left',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               plt.subplot(1,2,2)
               f_hist,bin = np.histogram(f_cent_lat,bins=np.arange(27.5,49.51,1.0))
               o_hist,bin = np.histogram(o_cent_lat,bins=np.arange(27.5,49.51,1.0))
               plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),align='center',width=1.0,color='red',edgecolor='white',linewidth=1,label=name)
               plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),align='center',width=1.0,color='None',edgecolor='black',linewidth=1,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel("Object centroid latitude (deg.)",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               mu_f = np.mean(f_cent_lat)
               mu_o = np.mean(o_cent_lat)
               std_f = np.std(f_cent_lat)
               std_o = np.std(o_cent_lat)
               med_f = np.median(f_cent_lat)
               med_o = np.median(o_cent_lat)
               plt.figtext(0.97,0.96,r"$\mu_f$={:4.1f};  $\mu_o$={:4.1f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.91,r"median$_f$={:4.1f};  median$_o$={:4.1f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.86,r"$\sigma_f$={:4.1f};  $\sigma_o$={:4.1f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.81,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.76,"CRPS={:5.3f}".format(cent_lat_crps),ha='right',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               image_file = "{}/{}_object_centroid_loc_hist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

               # area
               plt.figure(figsize=(5,5))
               plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.95)
               f_hist,bin = np.histogram(dx**2*f_area,bins=[144,180,270,360,450,540,630,720,810,900,1000,1200,1400,1600,1800,2000])
               o_hist,bin = np.histogram(dx**2*o_area,bins=[144,180,270,360,450,540,630,720,810,900,1000,1200,1400,1600,1800,2000])
               plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),width=bin[1:]-bin[:-1],align='center',color='red',edgecolor='white',linewidth=1,label=name)
               plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel(r"Object area [km$^{2}$]",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               plt.legend(loc=1,prop={'size':8})
               mu_f = dx**2*np.mean(f_area)
               mu_o = dx**2*np.mean(o_area)
               std_f = np.std(dx**2*f_area)
               std_o = np.std(dx**2*o_area)
               med_f = dx**2*np.median(f_area)
               med_o = dx**2*np.median(o_area)
               plt.figtext(0.93,0.83,r"$\mu_f$={:4.0f};  $\mu_o$={:4.0f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.78,r"median$_f$={:4.0f};  median$_o$={:4.0f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.73,r"$\sigma_f$={:4.0f};  $\sigma_o$={:4.0f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.68,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.63,"CRPS={:5.3f}".format(area_crps),ha='right',va='top',fontweight='bold',fontsize=9,bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               image_file = "{}/{}_object_area_hist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

               # length and width
               plt.figure(figsize=(9,5))
               plt.subplots_adjust(left=0.075,bottom=0.075,top=0.98,right=0.98,wspace=0.2)
               plt.subplot(1,2,1)
               f_hist,bin = np.histogram(dx*f_length,bins=[6,25,50,75,100,125,150,175,200,250,300,400,500,600,750,1000])
               o_hist,bin = np.histogram(dx*o_length,bins=[6,25,50,75,100,125,150,175,200,250,300,400,500,600,750,1000])
               plt.semilogy(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),'s-r',linewidth=2,label=name)
               plt.semilogy(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),'s-k',linewidth=2,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel("Object length [km]",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               plt.legend(loc=1,prop={'size':8})
               mu_f = dx*np.mean(f_length)
               mu_o = dx*np.mean(o_length)
               std_f = np.std(dx*f_length)
               std_o = np.std(dx*o_length)
               med_f = dx*np.median(f_length)
               med_o = dx*np.median(o_length)
               plt.figtext(0.48,0.83,r"$\mu_f$={:4.0f};  $\mu_o$={:4.0f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.48,0.78,r"median$_f$={:4.0f};  median$_o$={:4.0f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.48,0.73,r"$\sigma_f$={:4.0f};  $\sigma_o$={:4.0f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.48,0.68,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.48,0.63,"CRPS={:5.3f}".format(length_crps),ha='right',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               plt.subplot(1,2,2)
               f_hist,bin = np.histogram(dx*f_width,bins=[6,20,30,40,50,60,75,100,125,150,175,200,250,300])
               o_hist,bin = np.histogram(dx*o_width,bins=[6,20,30,40,50,60,75,100,125,150,175,200,250,300])
               plt.semilogy(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),'s-r',linewidth=2,label=name)
               plt.semilogy(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),'s-k',linewidth=2,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel("Object width [km]",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               mu_f = dx*np.mean(f_width)
               mu_o = dx*np.mean(o_width)
               std_f = np.std(dx*f_width)
               std_o = np.std(dx*o_width)
               med_f = dx*np.median(f_width)
               med_o = dx*np.median(o_width)
               plt.figtext(0.97,0.96,r"$\mu_f$={:3.0f};  $\mu_o$={:3.0f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.91,r"median$_f$={:3.0f};  median$_o$={:3.0f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.86,r"$\sigma_f$={:3.0f};  $\sigma_o$={:3.0f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.81,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.76,"CRPS={:5.3f}".format(width_crps),ha='right',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               image_file = "{}/{}_object_wid_len_hist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

               # aspect ratio
               plt.figure(figsize=(5,5))
               plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.98)
               f_hist,bin=np.histogram(f_aspect,bins=np.arange(0.0,1.01,0.1))
               o_hist,bin=np.histogram(o_aspect,bins=np.arange(0.0,1.01,0.1))
               plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),width=bin[1:]-bin[:-1],align='center',color='red',edgecolor='white',linewidth=1,label=name)
               plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlim(0,1)
               plt.xlabel("Object aspect ratio [width/length]",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               plt.legend(loc=1,prop={'size':8})
               mu_f = np.mean(f_aspect)
               mu_o = np.mean(o_aspect)
               std_f = np.std(f_aspect)
               std_o = np.std(o_aspect)
               med_f = np.median(f_aspect)
               med_o = np.median(o_aspect)
               plt.figtext(0.14,0.97,r"$\mu_f$={:5.3f};  $\mu_o$={:5.3f}".format(mu_f,mu_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.14,0.92,r"median$_f$={:5.3f};  median$_o$={:5.3f}".format(med_f,med_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.14,0.87,r"$\sigma_f$={:5.3f};  $\sigma_o$={:5.3f}".format(std_f,std_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.14,0.82,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.14,0.77,"CRPS={:5.3f}".format(aspect_crps),ha='left',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               image_file = "{}/{}_object_aspect_hist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

               # Complexity
               plt.figure(figsize=(5,5))
               plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.98)
               f_hist,bin=np.histogram(f_complexity,bins=np.arange(0.0,1.01,0.1))
               o_hist,bin=np.histogram(o_complexity,bins=np.arange(0.0,1.01,0.1))
               plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),width=bin[1:]-bin[:-1],align='center',color='red',edgecolor='white',linewidth=1,label=name)
               plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlim(0,1)
               plt.xlabel("Object complexity [-]",size=8)
               plt.ylabel("relative frequency",size=8)
               plt.tick_params(axis='both',labelsize=6)
               plt.legend(loc=1,prop={'size':8})
               mu_f = np.mean(f_complexity)
               mu_o = np.mean(o_complexity)
               std_f = np.std(f_complexity)
               std_o = np.std(o_complexity)
               med_f = np.median(f_complexity)
               med_o = np.median(o_complexity)
               plt.figtext(0.97,0.83,r"$\mu_f$={:5.3f};  $\mu_o$={:5.3f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.78,r"median$_f$={:5.3f};  median$_o$={:5.3f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.73,r"$\sigma_f$={:5.3f};  $\sigma_o$={:5.3f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.68,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.97,0.63,"CRPS={:5.3f}".format(complex_crps),ha='right',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               image_file = "{}/{}_object_complex_hist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

               # Intensity percentile values
               plt.figure(figsize=(5,5))
               plt.subplots_adjust(left=0.125,bottom=0.1,top=0.85,right=0.98)
               if True in np.isnan(f_pXX):
                print("Yeah, there are NaNs in f_pXX")
               f_hist,bin=np.histogram(f_pXX,bins=np.arange(25.0,70.1,5.0))
               o_hist,bin=np.histogram(o_pXX,bins=np.arange(25.0,70.1,5.0))
               plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(num_fcst_objs[ff]),width=bin[1:]-bin[:-1],align='center',color='lightsteelblue',edgecolor='white',linewidth=1,label=name)
               plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(num_obs_objs[ff]),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel("Object 95th percentile Value",size=8)
               plt.ylabel("Relative Frequency",size=8)
               plt.tick_params(axis='both',labelsize=7)
               plt.legend(loc=7,prop={'size':8})
               mu_f = np.mean(f_pXX)
               mu_o = np.mean(o_pXX)
               std_f = np.std(f_pXX)
               std_o = np.std(o_pXX)
               med_f = np.median(f_pXX)
               med_o = np.median(o_pXX)
               plt.figtext(0.95,0.80,r"$\mu_f$={:4.1f};  $\mu_o$={:4.1f}".format(mu_f,mu_o),ha='right',va='top',fontsize=8,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.95,0.75,r"median$_f$={:4.1f};  median$_o$={:4.1f}".format(med_f,med_o),ha='right',va='top',fontsize=8,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.95,0.70,r"$\sigma_f$={:4.1f};  $\sigma_o$={:4.1f}".format(std_f,std_o),ha='right',va='top',fontsize=8,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.95,0.65,r"N$_f$={};  N$_o$={}".format(num_fcst_objs[ff],num_obs_objs[ff]),ha='right',va='top',fontsize=8,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.95,0.60,"CRPS={:5.3f}".format(pXX_crps),ha='right',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
               plt.title("{}Z Object 95th Percentile Value".format(time_of_day), fontsize=10)
               image_file = "{}/{}_object_pXX_hist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

            # Pair interest
            if False:
               plt.figure(figsize=(5,5))
               plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
               hist,bins = np.histogram(pair_interest,bins=np.arange(0.0,1.01,0.05))
               plt.bar(bins[:-1],hist/float(len(pair_interest)),edgecolor='black',facecolor='red',width=0.05)
               plt.xticks(np.arange(0.0,1.01,0.1))
               plt.xlim(0,1)
               plt.grid(linestyle=":",color='0.75')
               plt.xlabel("Pair interest",fontsize=10)
               plt.ylabel("Relative frequency",fontsize=10)
               plt.tick_params(axis='both',labelsize=8)
               mu_f = np.mean(pair_interest)
               std_f = np.std(pair_interest)
               med_f = np.median(pair_interest)
               plt.figtext(0.93,0.88,r"$\mu$={:5.3f}".format(mu_f),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.83,"median={:5.3f}".format(med_f),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.78,r"$\sigma$={:5.3f}".format(std_f),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               plt.figtext(0.93,0.73,"N={}".format(len(pair_interest)),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
               image_file = "{}/{}_pair_interest_dist_r{}t{}_{}Z.png".format(img_dir,name,R,T,time_of_day)
               plt.savefig(image_file,dpi=120)
               plt.close()

         # save intermediate calculations to npz files for later re-use
         np.savez("{}/mode_metrics_r{}t{}_{}Z".format(img_dir,R,T,time_of_day),OTS=OTS[ff],MMI=MMI[ff],POD=total_pod[ff],FAR=total_far[ff],SR=total_sr[ff],CSI=total_csi[ff],
                  n_f_objs=num_fcst_objs[ff],n_o_objs=num_obs_objs[ff],area_crps=area_crps,length_crps=length_crps,width_crps=width_crps,aspect_crps=aspect_crps,
                  complex_crps=complex_crps,pXX_crps=pXX_crps,lon_crps=cent_lon_crps,lat_crps=cent_lat_crps,ncases=n_cases,
                  stm_mean_dist=mean_centroid_dist[ff,0],stm_med_dist=median_centroid_dist[ff,0],stm_std_dist=std_centroid_dist[ff,0],
                  all_mean_dist=mean_centroid_dist[ff,1],all_med_dist=median_centroid_dist[ff,1],all_std_dist=std_centroid_dist[ff,1],fbias=fbias,
                  gen_mean_dist=mean_centroid_dist[ff,2],gen_med_dist=median_centroid_dist[ff,2],gen_std_dist=std_centroid_dist[ff,2],
                  MMI_flag=standard_MMI,agg_start_date=args.idate,agg_end_date=args.edate)

         ff+=1 #increment independent loop counter so that numpy arrays can still be 0-indexed even if valid times do not start at 00Z

      # END valid time loop

      # Calculate final time-aggregated scores
      for d in range(0,len(agg_vhr_start)):
         if agg_hit[d] + agg_miss[d] > 0:
            agg_pod[d] = float(agg_hit[d]) / (agg_hit[d] + agg_miss[d])
         if agg_hit[d] + agg_fa[d] > 0:
            agg_sr[d] = float(agg_hit[d]) / (agg_hit[d] + agg_fa[d])
            agg_far[d] = float(agg_fa[d]) / (agg_hit[d] + agg_fa[d])
         if agg_hit[d] + agg_fa[d] + agg_miss[d] > 0:
            agg_csi[d] = float(agg_hit[d]) / (agg_hit[d] + agg_fa[d] + agg_miss[d])
         agg_mean_dist[d] = agg_dist_sum[d] / agg_dist_count[d]
         agg_OTS[d] = agg_ots_sum[d,0] / agg_ots_sum[d,1]
         exec("agg_MMI[{:d}] = np.median(agg_max_int_{:02d})".format(d,d))
         # Save scores to file for later plotting
         np.savez("{}/agg_scores_r{}t{}_{:02d}Z-{:02d}Z".format(img_dir,R,T,agg_vhr_start[d],agg_vhr_end[d]),
                  OTS=agg_OTS[d],MMI=agg_MMI[d],MMI_flag=standard_MMI,POD=agg_pod[d],FAR=agg_far[d],SR=agg_sr[d],CSI=agg_csi[d],
                  NF=agg_fcst_count[d],NO=agg_obs_count[d],mean_dist=agg_mean_dist[d],gen_dist=agg_gen_dist[d],obj_fbias=agg_fbias[d])

      # Sneak in plots of aggregated data
      print("Making plots of time-aggregated spatial frequency bias")
      for d in range(0,len(agg_vhr_start)):
         exec("f_hist2d,binx,biny = np.histogram2d(agg_f_cent_lon_{:02d},agg_f_cent_lat_{:02d},bins=[lon_bins,lat_bins])".format(d,d))
         exec("o_hist2d,binx,biny = np.histogram2d(agg_o_cent_lon_{:02d},agg_o_cent_lat_{:02d},bins=[lon_bins,lat_bins])".format(d,d))
         fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
         fig = plt.figure(figsize=figsize)
         ax = plt.subplots_adjust(left=0.01,right=0.95,bottom=0.05,top=0.97)
         m = Basemap(projection='lcc',width=proj_wid,height=proj_hgt,lat_0=clat,lon_0=clon,lat_1=38.5,resolution=res,area_thresh=min_area)
         m.drawlsmask(land_color='0.9',ocean_color='powderblue')
         m.drawcountries(linewidth=1,color='0.1')
         if True:
            m.drawcoastlines(linewidth=0.5,color='0.4')
         else:
            m.drawcounties(linewidth=0.5,color='0.6',linestyle='-')
         m.drawstates(linewidth=0.5,color='0.5')
         m.drawmapboundary(color=[0.25,0.01,0.02],linewidth=3)
         m.drawmeridians(np.arange(-130.0,-50.1,5.0),linewidth=0.1,labels=[0,0,0,1],labelstyle='E/W',yoffset=lbl_off_y,fontsize=8,dashes=[2,2])
         m.drawparallels(np.arange(20.0,55.1,5.0),linewidth=0.1,labels=[0,1,0,0],labelstyle='E/W',xoffset=lbl_off_x,fontsize=8,dashes=[2,2])
         x2d,y2d = m(lons_2d,lats_2d)
         fb_clevs_low = np.array([0,0.25,0.5,0.75,0.9])
         fb_clevs_high = np.array([1.1,1.25,1.5,2,2.5,3,3.5,4])
         fb_clevs = np.append(fb_clevs_low,fb_clevs_high)
         cm_low = mplc.get_cmap('Oranges_r',len(fb_clevs_low))
         cm_high = mplc.get_cmap('Purples',len(fb_clevs_high))
         cmap = np.vstack((cm_low(np.linspace(0,1,len(fb_clevs_low)-1)),np.array([0.9,1.0,0.9,1.0]),cm_high(np.linspace(0,1,len(fb_clevs_high)))))
         final_colors = ListedColormap(cmap)
         norm = BoundaryNorm(boundaries=fb_clevs,ncolors=len(fb_clevs),clip=True)
     #  plt.pcolormesh(x2d[:-1,:-1],y2d[:-1,:-1],fbias_hist2d,norm=norm,cmap=final_colors)
         plt.contourf(x2d[:-1,:-1],y2d[:-1,:-1],fbias_hist2d,levels=fb_clevs,colors=cmap,extend='max')
         cb = plt.colorbar(orientation='horizontal',fraction=0.05,aspect=50,pad=0.03,shrink=0.5,extend='max')
         cb.set_ticks(fb_clevs) # Frequency bias plot
         cb.set_label(r'Frequency bias ($N_f / N_o$)',fontsize=10)
         cb.ax.tick_params(labelsize=8)
         exec("_nf_ = len(agg_f_cent_lon_{:02d})".format(d))
       	 exec("_no_ = len(agg_o_cent_lon_{:02d})".format(d))
         min_sample = np.amin(o_hist2d)
         med_sample = np.median(o_hist2d)
         plt.figtext(0.75,0.22,r"$N_f$={}".format(_nf_) + "\n" + r"$N_o$={}".format(_no_) + "\n" + r"median obs bin size = {:.0f}".format(med_sample),
                     ha='left',va='top',fontsize=10,fontweight=500,bbox=dict(facecolor='white',edgecolor='black',boxstyle='round,pad=0.3',linewidth=1.5))
         image_file = "{}/{}_centroid_fbias_heatmap_r{}t{}_{:02d}Z-{:02d}Z_{}.png".format(img_dir,name,R,T,agg_vhr_start[d],agg_vhr_end[d],res_label)
         fig.savefig(image_file,dpi=120)
         plt.close()

      # Make plots of all-24-hour object attributes
      if True:
         total_fcst_objs = np.sum(num_fcst_objs)
         total_obs_objs = np.sum(num_obs_objs)
         mass_crps = calc_CRPS(f_mass,o_mass,mass_bins)
         area_crps = calc_CRPS(f_area,o_area,area_bins)
         aspect_crps = calc_CRPS(f_aspect,o_aspect,np.arange(0.0,1.01,0.01))
         complex_crps = calc_CRPS(f_complexity,o_complexity,np.arange(0.0,1.01,0.01))
         pXX_crps = calc_CRPS(f_pXX,o_pXX,pXX_bins)

         # Object area
         plt.figure(figsize=(5,5))
         plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.95)
         f_hist,bin = np.histogram(dx**2*all_f_area,bins=area_bins)
         o_hist,bin = np.histogram(dx**2*all_o_area,bins=area_bins)
         plt.semilogx(0.5*(bin[:-1]+bin[1:]),f_hist/float(total_fcst_objs),'r-x',markersize=6,label=name)
         plt.semilogx(0.5*(bin[:-1]+bin[1:]),o_hist/float(total_obs_objs),'k-x',markersize=6,label="MRMS")
         plt.grid(linestyle=":",color='0.75')
         plt.xlabel(r"Object area [km$^{2}$]",size=8)
         plt.ylabel("relative frequency",size=8)
         plt.tick_params(axis='both',labelsize=6)
         plt.legend(loc=1,prop={'size':8})
         mu_f = dx**2*np.mean(all_f_area)
         mu_o = dx**2*np.mean(all_o_area)
         std_f = np.std(dx**2*all_f_area)
         std_o = np.std(dx**2*all_o_area)
         med_f = dx**2*np.median(all_f_area)
         med_o = dx**2*np.median(all_o_area)
         plt.figtext(0.93,0.83,r"$\mu_f$={:4.0f};  $\mu_o$={:4.0f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.78,r"median$_f$={:4.0f};  median$_o$={:4.0f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.73,r"$\sigma_f$={:4.0f};  $\sigma_o$={:4.0f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.68,r"N$_f$={};  N$_o$={}".format(total_fcst_objs,total_obs_objs),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.63,"CRPS={:5.3f}".format(area_crps),ha='right',va='top',fontweight='bold',fontsize=9,bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
         image_file = "{}/{}_object_area_hist_r{}t{}_day.png".format(img_dir,name,R,T)
         plt.savefig(image_file,dpi=120)
         plt.close()

         # aspect ratio
         plt.figure(figsize=(5,5))
         plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.98)
         f_hist,bin=np.histogram(all_f_aspect,bins=np.arange(0.0,1.01,0.05))
         o_hist,bin=np.histogram(all_o_aspect,bins=np.arange(0.0,1.01,0.05))
         plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(total_fcst_objs),width=bin[1:]-bin[:-1],align='center',color='red',edgecolor='white',linewidth=1,label=name)
         plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(total_obs_objs),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
         plt.grid(linestyle=":",color='0.75')
         plt.xlim(0,1)
         plt.xlabel("Object aspect ratio [width/length]",size=8)
         plt.ylabel("relative frequency",size=8)
         plt.tick_params(axis='both',labelsize=6)
         plt.legend(loc=1,prop={'size':8})
         mu_f = np.mean(all_f_aspect)
         mu_o = np.mean(all_o_aspect)
         std_f = np.std(all_f_aspect)
         std_o = np.std(all_o_aspect)
         med_f = np.median(all_f_aspect)
         med_o = np.median(all_o_aspect)
         plt.figtext(0.14,0.97,r"$\mu_f$={:5.3f};  $\mu_o$={:5.3f}".format(mu_f,mu_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.14,0.92,r"median$_f$={:5.3f};  median$_o$={:5.3f}".format(med_f,med_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.14,0.87,r"$\sigma_f$={:5.3f};  $\sigma_o$={:5.3f}".format(std_f,std_o),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.14,0.82,r"N$_f$={};  N$_o$={}".format(total_fcst_objs,total_obs_objs),ha='left',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.14,0.77,"CRPS={:5.3f}".format(aspect_crps),ha='left',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
         image_file = "{}/{}_object_aspect_hist_r{}t{}_day.png".format(img_dir,name,R,T)
         plt.savefig(image_file,dpi=120)
         plt.close()

         # complexity
         plt.figure(figsize=(5,5))
         plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.98)
         f_hist,bin=np.histogram(all_f_complexity,bins=np.arange(0.0,1.01,0.05))
         o_hist,bin=np.histogram(all_o_complexity,bins=np.arange(0.0,1.01,0.05))
         plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(total_fcst_objs),width=bin[1:]-bin[:-1],align='center',color='red',edgecolor='white',linewidth=1,label=name)
         plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(total_obs_objs),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
         plt.grid(linestyle=":",color='0.75')
         plt.xlim(0,1)
         plt.xlabel("Object complexity [-]",size=8)
         plt.ylabel("relative frequency",size=8)
         plt.tick_params(axis='both',labelsize=6)
         plt.legend(loc=1,prop={'size':8})
         mu_f = np.mean(all_f_complexity)
         mu_o = np.mean(all_o_complexity)
         std_f = np.std(all_f_complexity)
         std_o = np.std(all_o_complexity)
         med_f = np.median(all_f_complexity)
         med_o = np.median(all_o_complexity)
         plt.figtext(0.97,0.83,r"$\mu_f$={:5.3f};  $\mu_o$={:5.3f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.78,r"median$_f$={:5.3f};  median$_o$={:5.3f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.73,r"$\sigma_f$={:5.3f};  $\sigma_o$={:5.3f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.68,r"N$_f$={};  N$_o$={}".format(total_fcst_objs,total_obs_objs),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.63,"CRPS={:5.3f}".format(complex_crps),ha='right',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
         image_file = "{}/{}_object_complex_hist_r{}t{}_day.png".format(img_dir,name,R,T)
         plt.savefig(image_file,dpi=120)
         plt.close()

         # percentile intensity values
         pbins = np.arange(0.0,100.1,5.0)
         plt.figure(figsize=(5,5))
         plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.98)
         f_hist,bin=np.histogram(all_f_pXX,bins=pXX_bins)
         o_hist,bin=np.histogram(all_o_pXX,bins=pXX_bins)
         plt.bar(0.5*(bin[:-1]+bin[1:]),f_hist/float(total_fcst_objs),width=bin[1:]-bin[:-1],align='center',color='red',edgecolor='white',linewidth=1,label=name)
         plt.bar(0.5*(bin[:-1]+bin[1:]),o_hist/float(total_obs_objs),width=1.0*(bin[1:]-bin[:-1]),align='center',color='None',edgecolor='black',linewidth=1,label="MRMS")
         plt.yscale('log')
         plt.grid(linestyle=":",color='0.75')
         if field == 'compref':
            plt.xlabel("Object 95th percentile value",size=8)
         elif field == 'precip':
            plt.xlabel("Object 99th percentile value",size=8)
         plt.ylabel("relative frequency",size=8)
         plt.tick_params(axis='both',labelsize=6)
         plt.legend(loc=1,prop={'size':8})
         mu_f = np.mean(all_f_pXX)
         mu_o = np.mean(all_o_pXX)
         std_f = np.std(all_f_pXX)
         std_o = np.std(all_o_pXX)
         med_f = np.median(all_f_pXX)
         med_o = np.median(all_o_pXX)
         plt.figtext(0.97,0.83,r"$\mu_f$={:4.1f};  $\mu_o$={:4.1f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.78,r"median$_f$={:4.1f};  median$_o$={:4.1f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.73,r"$\sigma_f$={:4.1f};  $\sigma_o$={:4.1f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.97,0.68,r"N$_f$={};  N$_o$={}".format(total_fcst_objs,total_obs_objs),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.14,0.77,"CRPS={:5.3f}".format(pXX_crps),ha='left',va='top',fontsize=9,fontweight='bold',bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
         image_file = "{}/{}_object_pXX_hist_r{}t{}_day.png".format(img_dir,name,R,T)
         plt.savefig(image_file,dpi=120)
         plt.close()

         # Object total mass
         plt.figure(figsize=(5,5))
         plt.subplots_adjust(left=0.125,bottom=0.1,top=0.98,right=0.95)
         f_hist,bin = np.histogram(all_f_mass,bins=mass_bins)
         o_hist,bin = np.histogram(all_o_mass,bins=mass_bins)
         plt.semilogx(0.5*(bin[:-1]+bin[1:]),f_hist/float(total_fcst_objs),'r-x',ms=6,label=name)
         plt.semilogx(0.5*(bin[:-1]+bin[1:]),o_hist/float(total_obs_objs),'k-x',ms=6,label="MRMS")
         plt.grid(linestyle=":",color='0.75')
         plt.xlabel(r"Object total mass (mm)",size=8)
         plt.ylabel("relative frequency",size=8)
         plt.tick_params(axis='both',labelsize=6)
         plt.legend(loc=1,prop={'size':8})
         mu_f = dx**2*np.mean(all_f_mass)
         mu_o = dx**2*np.mean(all_o_mass)
         std_f = np.std(dx**2*all_f_mass)
         std_o = np.std(dx**2*all_o_mass)
         med_f = dx**2*np.median(all_f_mass)
         med_o = dx**2*np.median(all_o_mass)
         plt.figtext(0.93,0.83,r"$\mu_f$={:4.0f};  $\mu_o$={:4.0f}".format(mu_f,mu_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.78,r"median$_f$={:4.0f};  median$_o$={:4.0f}".format(med_f,med_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.73,r"$\sigma_f$={:4.0f};  $\sigma_o$={:4.0f}".format(std_f,std_o),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.68,r"N$_f$={};  N$_o$={}".format(total_fcst_objs,total_obs_objs),ha='right',va='top',fontsize=7,bbox=dict(facecolor='white',edgecolor='black',linewidth=0.5))
         plt.figtext(0.93,0.63,"CRPS={:5.3f}".format(mass_crps),ha='right',va='top',fontweight='bold',fontsize=9,bbox=dict(facecolor='white',edgecolor='black',linewidth=1))
         image_file = "{}/{}_object_mass_hist_r{}t{}_day.png".format(img_dir,name,R,T)
         plt.savefig(image_file,dpi=120)
         plt.close()

   # End convolution theshold loop
# End convolution radii loop

WORKING ON RADIUS 1
WORKING ON THRESHOLD 1
WORKING ON TIME OF DAY 0600 UTC
looping over all 01-hour forecasts starting at 05Z...
looping over all 02-hour forecasts starting at 04Z...
looping over all 03-hour forecasts starting at 03Z...
There are a total of 3514 forecast objects and 1785 observation objects to evaluate over 39 cases at 0600 UTC
There were 21 matched object pairs representing the shape of a single intense thunderstorm evaluated at 0600 UTC


/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


3475.0
1779.0


<ipython-input-3-eee9e0b684e7>:515: RuntimeWarning: divide by zero encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
<ipython-input-3-eee9e0b684e7>:515: RuntimeWarning: invalid value encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)


WORKING ON TIME OF DAY 1800 UTC
looping over all 01-hour forecasts starting at 17Z...
looping over all 02-hour forecasts starting at 16Z...
looping over all 03-hour forecasts starting at 15Z...
There are a total of 5397 forecast objects and 2622 observation objects to evaluate over 44 cases at 1800 UTC
There were 147 matched object pairs representing the shape of a single intense thunderstorm evaluated at 1800 UTC


...

/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-3-eee9e0b684e7>:455: RuntimeWarning: divide by zero encountered in true_divide
  bias_by_size = 1.0*f_hist / o_hist
<ipython-input-3-eee9e0b684e7>:455: RuntimeWarning: invalid value encountered in true_divide
  bias_by_size = 1.0*f_hist / o_hist


2154.0
485.0


<ipython-input-3-eee9e0b684e7>:515: RuntimeWarning: divide by zero encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
<ipython-input-3-eee9e0b684e7>:515: RuntimeWarning: invalid value encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
<ipython-input-3-eee9e0b684e7>:825: RuntimeWarning: invalid value encountered in double_scalars
  agg_mean_dist[d] = agg_dist_sum[d] / agg_dist_count[d]
<ipython-input-3-eee9e0b684e7>:826: RuntimeWarning: invalid value encountered in double_scalars
  agg_OTS[d] = agg_ots_sum[d,0] / agg_ots_sum[d,1]
/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcou

Making plots of time-aggregated spatial frequency bias


/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/matplotlib/contour.py:1483: UserWarning: Warning: converting a masked element to nan.
  self.zmax = float(z.max())
/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/matplotlib/contour.py:1484: UserWarning: Warning: converting a masked element to nan.
  self.zmin = float(z.min())
<ipython-input-3-eee9e0b684e7>:838: RuntimeWarning: divide by zero encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
<ipython-input-3-eee9e0b684e7>:838: RuntimeWarning: invalid value encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
<ipython-input-3-eee9e0b684e7>:838: RuntimeWarning: invalid value encountered in true_divide
  fbias_hist2d = np.ma.masked_where(o_hist2d == 0,f_hist2d/o_hist2d)
/contrib/miniconda3/4.5.12/envs/pygraf/lib/python3.8/site-packages/matplotlib/contour.py:1483: UserWarning: Warning: converting a masked element to n

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.cm as mplc
import matplotlib.patheffects as pe

# Defaults
name = "TOD_all"
field = "compref"
img_dir = "/mnt/lfs4/BMC/wrfruc/nickel/mode_img/" + name + "/" + field + "/images/valid_time"
data_root = "/mnt/lfs4/BMC/wrfruc/nickel/mode_img/RRFS_CONUS_3km/compref/images/" #+ name + "/" + field
os.makedirs(img_dir, exist = True)
# In case, for whatever reason, you wish to narrow the display of valid hours...
ivhr = 0
evhr = 23
##### The above should generally always be 0 and 23
nhrs = evhr - ivhr + 1
thresh_mag = [25,30,35,40]
units = 'dBZ'
n_rad = 1
n_thresh = len(thresh_mag)
dx = 3.0 # grid spacing
# create color scheme for multiple convolution radius and threshold testing
cmp = mplc.get_cmap('coolwarm')
nq = np.linspace(0,1,n_rad*n_thresh)
colors = cmp(nq)
whiter_colors = np.zeros_like(colors)
for i in range(0,len(nq)):
   whiter_colors[i,:] = 0.5*(colors[i,:] + 1.0)
marks = ['o','^','D','s','+']
lines = ['-','--','-.',':','']
POD = np.zeros((n_rad,n_thresh,nhrs),dtype=np.float)
SR = np.zeros((n_rad,n_thresh,nhrs),dtype=np.float)
N_F = np.zeros(POD.shape,dtype=np.int)
N_O = np.zeros(POD.shape,dtype=np.int)
# The number of forecast cases is not (or should not, at least, be) a function of the convolution radius or threshold
n_cases = np.zeros(nhrs,dtype=np.int)
for vhr in range(ivhr,evhr+1):
   number_of_cases_data = np.load('{}/mode_metrics_r1t1_{:02d}Z.npz'.format(data_root,vhr))
   v = vhr - ivhr
   n_cases[v] = number_of_cases_data['ncases']

for R in range(1,n_rad+1):
   for T in range(1,n_thresh+1):

      if n_rad > 1:
         plot_label = "R{}-T{} {}".format(R,thresh_mag[T-1],units)
      else:
         plot_label = "{} {}".format(thresh_mag[T-1],units)

      color_number = n_thresh*(R-1) + T - 1
      CSI = np.zeros(nhrs,dtype=np.float)
      FAR = np.zeros(nhrs,dtype=np.float)
      MMI = np.zeros(nhrs,dtype=np.float)
      OTS = np.zeros(nhrs,dtype=np.float)
      area_CRPS = np.zeros(nhrs,dtype=np.float)
      aspect_CRPS = np.zeros(nhrs,dtype=np.float)
      complex_CRPS = np.zeros(nhrs,dtype=np.float)
      length_CRPS = np.zeros(nhrs,dtype=np.float)
      width_CRPS = np.zeros(nhrs,dtype=np.float)
      p95_CRPS = np.zeros(nhrs,dtype=np.float)
      lat_CRPS = np.zeros(nhrs,dtype=np.float)
      lon_CRPS = np.zeros(nhrs,dtype=np.float)
      obj_fbias = np.zeros(nhrs,dtype=np.float)
      mean_dist_stm = np.zeros(nhrs,dtype=np.float)
      med_dist_stm = np.zeros(nhrs,dtype=np.float)
      std_dist_stm = np.zeros(nhrs,dtype=np.float)
      mean_dist_all = np.zeros(nhrs,dtype=np.float)
      med_dist_all = np.zeros(nhrs,dtype=np.float)
      std_dist_all = np.zeros(nhrs,dtype=np.float)
      mean_dist_gen = np.zeros(nhrs,dtype=np.float)
      med_dist_gen = np.zeros(nhrs,dtype=np.float)
      std_dist_gen = np.zeros(nhrs,dtype=np.float)

      for vhr in range(ivhr,evhr+1):
         v = vhr - ivhr
         data = np.load('{}/mode_metrics_r{}t{}_{:02d}Z.npz'.format(data_root,R,T,vhr))

         CSI[v] = data['CSI']
       	 POD[R-1,T-1,v] = data['POD']
       	 FAR[v] = data['FAR']
       	 SR[R-1,T-1,v] = data['SR']
       	 MMI[v] = data['MMI']
       	 OTS[v] = data['OTS']
         N_F[R-1,T-1,v] = data['n_f_objs']
         N_O[R-1,T-1,v] = data['n_o_objs']
         area_CRPS[v] = data['area_crps']
       	 width_CRPS[v] = data['width_crps']
       	 length_CRPS[v] = data['length_crps']
       	 aspect_CRPS[v] = data['aspect_crps']
       	 complex_CRPS[v] = data['complex_crps']
       	 p95_CRPS[v] =	data['p95_crps']
         lon_CRPS[v] = data['lon_crps']
         lat_CRPS[v] = data['lat_crps']
         obj_fbias[v] = data['fbias']
         mean_dist_stm[v] = data['stm_mean_dist']
         med_dist_stm[v] = data['stm_med_dist']
         std_dist_stm[v] = data['stm_std_dist']
         mean_dist_all[v] = data['all_mean_dist']
         med_dist_all[v] = data['all_med_dist']
         std_dist_all[v] = data['all_std_dist']
         mean_dist_gen[v] = data['gen_mean_dist']
         med_dist_gen[v] = data['gen_med_dist']
         std_dist_gen[v] = data['gen_std_dist']
         obj_fbias[v] = np.where(N_O[R-1,T-1,v] == 0.,np.nan,1.0*N_F[R-1,T-1,v]/N_O[R-1,T-1,v])

      plt.figure(1,figsize=(5,5))
      plt.subplots_adjust(left=0.1,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),OTS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(2,figsize=(5,5))
      plt.subplots_adjust(left=0.1,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),MMI,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(6,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),area_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(7,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),width_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(8,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),length_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(9,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),aspect_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(10,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),complex_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(11,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),p95_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(12,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),lon_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(13,figsize=(5,5))
      plt.subplots_adjust(left=0.15,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),lat_CRPS,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(14,figsize=(5,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.fill_between([ivhr,evhr],np.ones(2),[10.,10.],facecolor=[1.0,1.0,0.33,0.05],linestyle='None')
      plt.fill_between([ivhr,evhr],[0.0,0.0],np.ones(2),facecolor=[0.75,0.37,1.0,0.05],linestyle='None')
      t1 = plt.text(3,3.25,"OVERFORECAST",ha='left',va='top',color=[1.0,1.0,0.33],fontsize=12,fontweight=700,bbox=dict(facecolor='white',linewidth=2,edgecolor=[0.75,0.75,0.25],boxstyle='round,pad=0.5'))
      t2 = plt.text(12,0.50,"UNDERFORECAST",ha='center',va='center',color=[0.75,0.37,1.0],fontsize=12,fontweight=700,bbox=dict(facecolor='white',linewidth=2,edgecolor=[0.56,0.278,0.75],boxstyle='round,pad=0.5'))
      t1.set_path_effects([pe.Stroke(linewidth=2,foreground='0.25'),pe.Normal()])
      t2.set_path_effects([pe.Stroke(linewidth=2,foreground='0.1'),pe.Normal()])
      plt.plot([ivhr,evhr],[1.,1.],'k-',linewidth=2)
      plt.plot(range(ivhr,evhr+1),obj_fbias,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      mean_dist_stm[mean_dist_stm == -999.] = np.nan
      med_dist_stm[med_dist_stm == -999.] = np.nan
      std_dist_stm[mean_dist_stm == -999.] = np.nan
      mean_dist_all[mean_dist_all == -999.] = np.nan
      med_dist_all[med_dist_all == -999.] = np.nan
      std_dist_all[mean_dist_all == -999.] = np.nan
      mean_dist_gen[mean_dist_gen == -999.] = np.nan
      med_dist_gen[med_dist_gen == -999.] = np.nan
      std_dist_gen[mean_dist_gen == -999.] = np.nan
      plt.figure(15,figsize=(6,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
#      plt.plot(range(ivhr,evhr+1),mean_dist_stm,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)
      plt.errorbar(range(ivhr,evhr+1),mean_dist_stm,yerr=std_dist_stm,fmt='.-',markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(16,figsize=(6,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),med_dist_stm,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(17,figsize=(6,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),mean_dist_all,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(18,figsize=(6,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),med_dist_all,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(19,figsize=(6,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),mean_dist_gen,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(20,figsize=(6,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),med_dist_gen,linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(21,figsize=(5,5))
      plt.subplots_adjust(left=0.1,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),np.where(N_F[R-1,T-1,:] == 0,np.nan,1.*N_F[R-1,T-1,:]/n_cases),linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

      plt.figure(22,figsize=(5,5))
      plt.subplots_adjust(left=0.1,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),np.where(N_O[R-1,T-1,:] == 0,np.nan,1.*N_O[R-1,T-1,:]/n_cases),linestyle=lines[R-1],marker=marks[T-1],markersize=4,color=colors[color_number,:],label=plot_label)

   # End convolution theshold loop
# End convolution radii loop

#print("Data range verified (YYYYMMDDHH of initialization): {} to {}".format(data['agg_start_date'],data['agg_end_date']))

# Make sure these lists correspond to each other and to the order of the plotting above
if data['MMI_flag']:
   MMI_title = "MMI_std"
   MMI_name = "standard MMI"
   print("Plotting standard MMI calculation")
else:
   MMI_title = "MMI_alt"
   MMI_name = "alternate MMI"
   print("Plotting alternative MMI calculation")

fig_titles = ["OTS",MMI_title,"","","","CRPS_area","CRPS_width","CRPS_length","CRPS_aspect","CRPS_complex","CRPS_p95","CRPS_lon","CRPS_lat","object_fbias",
              "mean_distance_stm","median_distance_stm","mean_distance_all","median_distance_all","mean_distance_gen","median_distance_gen","fcst_object_count","obs_object_count"]
fig_y_axis = ["OTS",MMI_name,"","","",
              "area distribution CRPS","width distribution CRPS","length distribution CRPS","aspect-ratio distribution CRPS","complexity CRPS","95th pct. distribution CRPS","centroid longitude CRPS","centroid latitude CRPS",
              r"object frequency bias ($N_{f} / N_{o}$)","mean distance between matched storm objects","median distance between matched storm objects","mean distance between matched objects","median distance between matched objects",
              "generalized mean distance between objects","generalized median distance between objects","average number of forecast objects per case","average number of observation objects per case"]
for n in range(1,23):
 if n <= 2 or n >= 6:
  plt.figure(n)
  plt.grid(linestyle=":",color='0.75')
  plt.xticks(range(0,24,3))
  plt.xlim(ivhr-1,evhr+1)
  if n <= 14:
   if "bias" in fig_titles[n-1]:
    plt.xlim(-0.5,23.5)
    plt.yticks([0.0,0.5,0.75,0.9,1.0,1.1,1.25,1.5,2,2.5,3,3.5,4])
    plt.ylim(0,3.5)
   elif "MMI" in fig_titles[n-1]:
    plt.yticks(np.arange(0.00,1.01,0.05))
    plt.ylim(0.0,0.7)
   elif "OTS" in fig_titles[n-1]:
    plt.ylim(0.25,0.9)
    plt.yticks(np.arange(0.25,0.91,0.05))
  elif n >= 15 and n<= 20:
   plt.ylim(0,60)
  plt.xlabel("valid time (UTC)",size=8)
  plt.ylabel(fig_y_axis[n-1],size=8)
  plt.tick_params(axis='both',labelsize=6)
  plt.legend(loc=0,prop={'size':8},ncol=n_rad)
  image_file = "{}/{}_{}_{}.png".format(img_dir,name,field,fig_titles[n-1])
  plt.savefig(image_file,dpi=120)
  plt.close(n)

def performance_diagram_window(flag):
# Settings to control the window of the performance diagram to be displayed
   if flag:
      plt.xlim(0,1)
      plt.ylim(0,1)
      plt.xticks(np.arange(0.0,1.01,0.1))
      plt.yticks(np.arange(0.0,1.01,0.1))
   else:
      plt.xticks(np.arange(0.0,1.01,0.05))
      plt.yticks(np.arange(0.0,1.01,0.05))
      plt.xlim(0.2,0.7)
      plt.ylim(0.5,0.9)

# Roebber performance diagram containing ALL data
plt.figure(figsize=(5,5))
plt.subplots_adjust(left=0.1,right=0.94,bottom=0.1,top=0.97)
for csi in np.arange(0.1,0.91,0.1):
   x = np.linspace(0.01,1.0,100)
   y = np.zeros(x.shape,dtype=np.float)
   for i in range(0,len(x)):
      y[i] = 1.0 / (1/csi - 1/x[i] + 1)
      if y[i] < 0.0 or y[i] > y[np.maximum(i-1,0)]:
         y[i] = 1.0
   plt.plot(x,y,'--',linewidth=0.5,color='0.7')
   plt.text(x[95],y[95],"{:3.1f}".format(csi),fontsize=6,color='0.7',ha='center',va='center',bbox=dict(facecolor='white',edgecolor='None',pad=1))
performance_diagram_window(False)
for bias in [0.25,0.5,0.75,0.9,1.0,1.1,1.25,1.5,2,3,4,5]:
   plt.plot([0,1],[0,bias],'-',linewidth=0.5,color='0.7')
   if bias < 1.0:
      plt.text(1.0,bias," {:4.2f}".format(bias),fontsize=6,color='0.7',ha='left')
   else:
      plt.text(1.0/bias,1.0,"{:4.2f}".format(bias),fontsize=6,color='0.7',va='bottom')
plt.plot([0,1],[0,1],'-',linewidth=1.5,color='0.3')
for R in range(1,n_rad+1):
   for T in range(1,n_thresh+1):
      if n_rad > 1:
         plot_label = "R{}-T{} {}".format(R,thresh_mag[T-1],units)
      else:
         plot_label = "{} {}".format(thresh_mag[T-1],units)
      color_number = n_thresh*(R-1) + T - 1
      plt.plot(SR[R-1,T-1,:],POD[R-1,T-1,:],marker=marks[T-1],markersize=4,mfc=colors[color_number],color=colors[color_number,:],label=plot_label)
for R in range(1,n_rad+1):
   for T in range(1,n_thresh+1):
      color_number = n_thresh*(R-1) + T - 1
      plt.plot(SR[R-1,T-1,0],POD[R-1,T-1,0],marker=marks[T-1],markersize=5,mfc='yellow',mew=0.5,mec='k')
      plt.plot(SR[R-1,T-1,12],POD[R-1,T-1,12],marker=marks[T-1],markersize=5,mfc='orange',mew=0.5,mec='k')
      plt.plot(SR[R-1,T-1,23],POD[R-1,T-1,23],marker=marks[T-1],markersize=5,mfc='sienna',mew=0.5,mec='k')
plt.xlabel("success ratio (1-FAR)",size=8)
plt.ylabel("POD",size=8)
plt.tick_params(axis='both',labelsize=6)
plt.legend(loc='lower right',prop={'size':8},ncol=1,fancybox=True)
image_file = "{}/{}_{}_object_performance_diagram_all.png".format(img_dir,name,field)
plt.savefig(image_file,dpi=120)
plt.close()

# Roebber performance diagram by valid hour
for vhr in range(ivhr,evhr+1):
   v = vhr - ivhr
   plt.figure(6,figsize=(4,4))
   plt.subplots_adjust(left=0.12,right=0.94,bottom=0.1,top=0.97)
   for csi in np.arange(0.1,0.91,0.1):
      x = np.linspace(0.01,1.0,100)
      y = np.zeros(x.shape,dtype=np.float)
      for i in range(0,len(x)):
         y[i] = 1.0 / (1/csi - 1/x[i] + 1)
         if y[i] < 0.0 or y[i] > y[np.maximum(i-1,0)]:
            y[i] = 1.0
      plt.plot(x,y,'--',linewidth=0.5,color='0.7')
      plt.text(x[95],y[95],"{:3.1f}".format(csi),fontsize=6,color='0.7',ha='center',va='center',bbox=dict(facecolor='white',edgecolor='None',pad=1))
   performance_diagram_window(True)
   for bias in [0.25,0.5,0.75,0.9,1.0,1.1,1.25,1.5,2,3,4,5]:
      plt.plot([0,1],[0,bias],'-',linewidth=0.5,color='0.7')
      if bias < 1.0:
         plt.text(1.0,bias," {:4.2f}".format(bias),fontsize=6,color='0.7',ha='left')
      else:
         plt.text(1.0/bias,1.0,"{:4.2f}".format(bias),fontsize=6,color='0.7',va='bottom')
   plt.plot([0,1],[0,1],'-',linewidth=1.5,color='0.3')
   for R in range(1,n_rad+1):
      for T in range(1,n_thresh+1):
         if n_rad > 1:
            plot_label = "R{}-T{} {}".format(R,thresh_mag[T-1],units)
         else:
            plot_label = "{} {}".format(thresh_mag[T-1],units)
         color_number = n_thresh*(R-1) + T - 1
         plt.plot(SR[R-1,T-1,v],POD[R-1,T-1,v],marker=marks[T-1],markersize=4,mfc=colors[color_number],color=colors[color_number,:],label=plot_label)
   plt.xlabel("success ratio (1-FAR)",size=8)
   plt.ylabel("POD",size=8)
   plt.tick_params(axis='both',labelsize=6)
   plt.legend(loc='lower right',prop={'size':8},ncol=1,fancybox=True)
   image_file = "{}/{}_{}_object_performance_diagram_{:02d}Z.png".format(img_dir,name,field,vhr)
   plt.savefig(image_file,dpi=120)
   plt.close(6)

# Roebber performance diagram by MODE configuration
for R in range(1,n_rad+1):
   for T in range(1,n_thresh+1):
      color_number = n_thresh*(R-1) + T - 1
      plt.figure(6,figsize=(4,4))
      plt.subplots_adjust(left=0.12,right=0.94,bottom=0.1,top=0.97)
      for csi in np.arange(0.1,0.91,0.1):
         x = np.linspace(0.01,1.0,100)
         y = np.zeros(x.shape,dtype=np.float)
         for i in range(0,len(x)):
            y[i] = 1.0 / (1/csi - 1/x[i] + 1)
            if y[i] < 0.0 or y[i] > y[np.maximum(i-1,0)]:
               y[i] = 1.0
         plt.plot(x,y,'--',linewidth=0.5,color='0.7')
         plt.text(x[95],y[95],"{:3.1f}".format(csi),fontsize=6,color='0.7',ha='center',va='center',bbox=dict(facecolor='white',edgecolor='None',pad=1))
      performance_diagram_window(False)
      for bias in [0.25,0.5,0.75,0.9,1.0,1.1,1.25,1.5,2,3,4,5]:
         plt.plot([0,1],[0,bias],'-',linewidth=0.5,color='0.7')
         if bias < 1.0:
            plt.text(1.0,bias," {:4.2f}".format(bias),fontsize=6,color='0.7',ha='left')
         else:
            plt.text(1.0/bias,1.0,"{:4.2f}".format(bias),fontsize=6,color='0.7',va='bottom')
      plt.plot([0,1],[0,1],'-',linewidth=1.5,color='0.3')
      plt.plot(SR[R-1,T-1,:],POD[R-1,T-1,:],marker=marks[T-1],linestyle='-',linewidth=0.5,markersize=4,mfc=colors[color_number],color=colors[color_number,:])
      plt.text(SR[R-1,T-1,0]+0.01,POD[R-1,T-1,0]+0.01,"{:02d}Z".format(ivhr),fontsize=5,ha='left',va='baseline')
      plt.text(SR[R-1,T-1,12-ivhr]+0.01,POD[R-1,T-1,12-ivhr]+0.01,"{:02d}Z".format(12),fontsize=5,ha='left',va='baseline')
      plt.text(SR[R-1,T-1,evhr-ivhr]+0.01,POD[R-1,T-1,evhr-ivhr]+0.01,"{:02d}Z".format(evhr),fontsize=5,ha='left',va='baseline')
      plt.xlabel("success ratio (1-FAR)",size=8)
      plt.ylabel("POD",size=8)
      plt.tick_params(axis='both',labelsize=6)
      image_file = "{}/{}_{}_object_performance_diagram_r{}t{}.png".format(img_dir,name,field,R,T)
      plt.savefig(image_file,dpi=120)
      plt.close(6)

      # Object counts
      plt.figure(10,figsize=(5,5))
      plt.subplots_adjust(left=0.125,right=0.98,bottom=0.1,top=0.98)
      plt.plot(range(ivhr,evhr+1),N_F[R-1,T-1,:]/(1.*n_cases),'-x',linewidth=2,markersize=4,color=colors[color_number,:],label=name)
      plt.plot(range(ivhr,evhr+1),N_O[R-1,T-1,:]/(1.*n_cases),'k-x',linewidth=3,markersize=4,label="MRMS")
      plt.xticks(range(0,37,3))
      plt.grid(linestyle=":",color='0.75')
      plt.xlim(ivhr-1,evhr+1)
      plt.xlabel("Valid time (UTC)",size=8)
      plt.ylabel("Case-averaged object count",size=8)
      plt.tick_params(axis='both',labelsize=6)
      plt.legend(loc=0,prop={'size':8})
      image_file = "{}/{}_{}_object_count_r{}t{}.png".format(img_dir,name,field,R,T)
      plt.savefig(image_file,dpi=120)
      plt.close(10)